<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# GitHub - Send contributor activity on slack
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/GitHub/GitHub_Reopen_issue.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Reopen+issue:+Error+short+description">Bug report</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #github #activity #update #api #snippet #operations #slack 

**Author:** [Benjamin Filly](https://www.linkedin.com/in/benjamin-filly-05427727a/)

**Description:** This notebook explains how to send the GitHub activity on Slack

**References:**
- [GitHub REST API Documentation](https://docs.github.com/en/rest/issues/issues?apiVersion=2022-11-28#update-an-issue)

## Input

### Import libraries

In [1]:
import os
import json
from datetime import datetime
from github import Github
# from slack_sdk import WebClient
# from slack_sdk.errors import SlackApiError
import naas
import pandas as pd
import requests

### Setup Variables
- `token`: [GitHub token](https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line)
- `owner`: owner of the repository
- `repo`: name of the repository
- `issue_number`: number of the issue

In [2]:
# Input variables
github_token = naas.secret.get("GITHUB_TOKEN") or "GITHUB_TOKEN"
repo_name = "jupyter-naas/awesome-notebooks" #example
contributor_profile = "NAME"
slack_bot_token = naas.secret.get("SLACK_BOT_TOKEN") or "SLACK_TOKEN"
slack_chat = "<naas-templates-contributors>"

## Model

### Get PRs

In [3]:
# Connect to the GitHub API
g = Github(github_token)
# Get the repository
repo = g.get_repo(repo_name)
# Get the closed PR
pull_requests = repo.get_pulls(state="open")

# Print the closed PR
print("✅ Pull Requests fetched:", pull_requests.totalCount)
data = []
for index, pr in enumerate(pull_requests):
    # Init
    assignee_login = None
    assignee = pr.raw_data.get("assignee")
    if assignee:
        assignee_login = assignee.get("login")
    tmp = {
        "title": pr.title,
        "number": pr.number,
        "url": pr.raw_data.get("url"),
        "assignee": assignee_login,
        "created_at": pr.raw_data.get("created_at"),
        "updated_at": pr.raw_data.get("updated_at"),
    }
    data.append(tmp)

df_pr = pd.DataFrame(data)
df_pr.head(3)

✅ Pull Requests fetched: 26


,title,number,url,assignee,created_at,updated_at
0,feat: convert multiple units,1960,https://api.github.com/repos/jupyter-naas/awes...,Benjifilly,2023-06-21T11:59:56Z,2023-06-21T12:00:12Z
1,feat: GitHub - List branches,1955,https://api.github.com/repos/jupyter-naas/awes...,Benjifilly,2023-06-20T09:27:49Z,2023-06-21T12:17:53Z
2,feat: Naas - Create templates using MyChatGPT,1947,https://api.github.com/repos/jupyter-naas/awes...,FlorentLvr,2023-06-16T07:36:06Z,2023-06-20T09:21:59Z


### Filter dataframe on contributor
Do not forget to use .reset_index(drop=True)

In [4]:
contributor_filtered_df = df_pr[df_pr['assignee'] == contributor_profile].reset_index(drop=True)
contributor_filtered_df

,title,number,url,assignee,created_at,updated_at
0,feat: convert multiple units,1960,https://api.github.com/repos/jupyter-naas/awes...,Benjifilly,2023-06-21T11:59:56Z,2023-06-21T12:00:12Z
1,feat: GitHub - List branches,1955,https://api.github.com/repos/jupyter-naas/awes...,Benjifilly,2023-06-20T09:27:49Z,2023-06-21T12:17:53Z
2,feat: Pandas - Check if column is in date format,1897,https://api.github.com/repos/jupyter-naas/awes...,Benjifilly,2023-06-13T07:54:22Z,2023-06-19T09:32:08Z
3,feat: Deezer - Get listening history,1888,https://api.github.com/repos/jupyter-naas/awes...,Benjifilly,2023-06-09T09:41:43Z,2023-06-12T09:49:08Z
4,feat: Request - Handling Errors and Exceptions,1886,https://api.github.com/repos/jupyter-naas/awes...,Benjifilly,2023-06-09T09:33:32Z,2023-06-19T13:56:35Z


### Loop on PR to get comments to assign "in review" or "wip"

In [5]:
pd.set_option('display.max_colwidth', None)
prs_in_review = []
prs_work_in_progress = []

for index, row in contributor_filtered_df.iterrows():
    pr_number = row["number"]
    
    pr = next((pr for pr in pull_requests if pr.number == pr_number), None)
    
    if pr:
        comments = pr.get_issue_comments()
        
        if comments.totalCount > 0:
            last_comment = comments[0].body.lower()
            
            if "ready to review" in last_comment:
                prs_in_review.append(pr)
            else:
                prs_work_in_progress.append(pr)
        else:
            prs_work_in_progress.append(pr)

df_in_review = pd.DataFrame(columns=["title", "number", "url", "assignee", "created_at", "updated_at"])
for pr in prs_in_review:
    df_in_review = df_in_review.append({
        "title": pr.title,
        "number": pr.number,
        "url": pr.html_url,
        "assignee": pr.assignee.login if pr.assignee else None,
        "created_at": pr.created_at,
        "updated_at": pr.updated_at
    }, ignore_index=True)

df_work_in_progress = pd.DataFrame(columns=["title", "number", "url", "assignee", "created_at", "updated_at"])
for pr in prs_work_in_progress:
    df_work_in_progress = df_work_in_progress.append({
        "title": pr.title,
        "number": pr.number,
        "url": pr.html_url,
        "assignee": pr.assignee.login if pr.assignee else None,
        "created_at": pr.created_at,
        "updated_at": pr.updated_at
    }, ignore_index=True)

print("✅ Pull Requests in Review:")
print(df_in_review)
print("--------------------------")
print("👨‍💻 Pull Requests Work in Progress:")
print(df_work_in_progress)

✅ Pull Requests in Review:
                                              title number  \
0                      feat: convert multiple units   1960   
1  feat: Pandas - Check if column is in date format   1897   

                                                           url    assignee  \
0  https://github.com/jupyter-naas/awesome-notebooks/pull/1960  Benjifilly   
1  https://github.com/jupyter-naas/awesome-notebooks/pull/1897  Benjifilly   

           created_at          updated_at  
0 2023-06-21 11:59:56 2023-06-21 12:00:12  
1 2023-06-13 07:54:22 2023-06-19 09:32:08  
--------------------------
👨‍💻 Pull Requests Work in Progress:
                                            title number  \
0                    feat: GitHub - List branches   1955   
1            feat: Deezer - Get listening history   1888   
2  feat: Request - Handling Errors and Exceptions   1886   

                                                           url    assignee  \
0  https://github.com/jupyter-naas/aw

### Get issues

In [6]:
new_issues = []

for index, row in contributor_filtered_df.iterrows():
    contributor = row["assignee"]
    issues = repo.get_issues(state="open", assignee=contributor)
    
    for issue in issues:
        if not any(pr.number == issue.number for pr in pull_requests):
            new_issues.append(issue)
new_issues = list({issue.number: issue for issue in new_issues}.values())

# Print the new issues
print("New Issues:")
for issue in new_issues:
    print(f"Issue Title: {issue.title}")
    print(f"Issue Number: {issue.number}")
    print(f"Issue URL: {issue.html_url}")
    print("--------------------------")


New Issues:
Issue Title: Python - Convert multiples units
Issue Number: 1959
Issue URL: https://github.com/jupyter-naas/awesome-notebooks/issues/1959
--------------------------
Issue Title: GitHub - List branches
Issue Number: 1954
Issue URL: https://github.com/jupyter-naas/awesome-notebooks/issues/1954
--------------------------
Issue Title: Slack - Send blocks to channel
Issue Number: 1950
Issue URL: https://github.com/jupyter-naas/awesome-notebooks/issues/1950
--------------------------
Issue Title: GitHub - Send contributor activity on Slack
Issue Number: 1932
Issue URL: https://github.com/jupyter-naas/awesome-notebooks/issues/1932
--------------------------
Issue Title: Pandas - Check if column is in date format
Issue Number: 1896
Issue URL: https://github.com/jupyter-naas/awesome-notebooks/issues/1896
--------------------------
Issue Title: Deezer - Get listening history
Issue Number: 1887
Issue URL: https://github.com/jupyter-naas/awesome-notebooks/issues/1887
------------------

### Create Slack block

### Replace variables in Slack block

## Output

### Send message on Slack

In [ ]:
SLACK.send(SLACK_CHANNEL, text=None, blocks=blocks)